In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('mordred_desc.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 3227 entries, Yield to mZagreb2.1
dtypes: bool(4), float64(1675), int64(837), object(711)
memory usage: 1.7+ MB


In [3]:
df = df[df.columns.difference(df.select_dtypes(include=['object']).columns)]
df = df[df.columns.difference(df.select_dtypes(include=['bool']).columns)]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 2512 entries, AATS0Z to piPC9.1
dtypes: float64(1675), int64(837)
memory usage: 1.3 MB


In [4]:
#Preparing

# delete variables with zero variance
Var0Variable = np.where( df.var(axis=0) == 0 )
if len(Var0Variable[0]) == 0:
    print( "There is no variable with zero variance." )
    print( "" )
else:
    print( "There were {0} variables with zero variance".format(len(Var0Variable[0])))
    print( "the number of variable is: {0}".format(Var0Variable[0]) )
    print( "remove them" )
    print( "" )
    #df_var0 = np.delete(df, Var0Variable, 1)    
    df_var0 = df.drop(df.columns[Var0Variable], axis=1)

#removal of columns which data has all 0
def remove_all_zero_col(df):
    df = df.copy()
    for col in df.columns:
        if (df[col] == 0).all():
            df.drop(col, axis=1, inplace=True)
    return df
df_var0 = remove_all_zero_col(df_var0)
df_var0.info()

There were 678 variables with zero variance
the number of variable is: [ 264  290  294  438  460  476  478  480  482  484  486  488  490  492
  494  496  690  692  694  696  698  700  702  704  706  708  710  712
  800  802  804  805  806  809  810  812  813  814  816  838  842  860
  879  880  912 1029 1030 1040 1042 1044 1046 1062 1063 1073 1075 1077
 1079 1242 1246 1254 1268 1270 1311 1312 1313 1314 1315 1316 1317 1318
 1319 1320 1322 1325 1326 1327 1328 1329 1330 1332 1333 1334 1335 1336
 1337 1338 1340 1341 1342 1343 1344 1345 1346 1347 1348 1350 1351 1352
 1353 1354 1355 1356 1357 1358 1360 1364 1366 1367 1368 1369 1370 1371
 1372 1373 1374 1375 1376 1379 1380 1381 1382 1383 1384 1385 1386 1387
 1388 1389 1390 1391 1392 1393 1394 1395 1396 1398 1399 1400 1401 1402
 1403 1404 1407 1408 1409 1410 1411 1412 1413 1414 1415 1416 1417 1418
 1419 1420 1421 1422 1423 1424 1425 1426 1427 1428 1429 1430 1431 1432
 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442 1443 1444 1446 1447
 1448 

In [5]:
#Building descriptors
X = df_var0.drop(columns=['Yield'])

print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df_var0['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
      AATS0Z   AATS0Z.1  AATS0are  AATS0are.1    AATS0d  AATS0d.1   AATS0dv  \
0  25.000000  38.153846  6.502857    6.721046  3.000000  2.076923  8.428571   
1  25.000000  60.000000  6.502857    7.930514  3.000000  2.142857  8.428571   
2  25.000000  36.000000  6.502857    6.257725  3.000000  2.937500  8.428571   
3  25.000000  59.333333  6.502857    7.210600  3.000000  1.500000  8.428571   
4  22.823529  38.153846  6.292353    6.721046  2.941176  2.076923  7.411765   

   AATS0dv.1      AATS0i    AATS0i.1  ...     piPC4   piPC4.1    piPC5  \
0   8.476733  160.083901  170.333600  ...  3.803881  0.693147  4.20189   
1  12.313933  160.083901  174.439640  ...  3.803881  0.000000  4.20189   
2   7.637346  160.083901  158.503934  ...  3.803881  3.944006  4.20189   
3  10.185185  160.083901  172.604366  ...  3.803881  0.000000  4.20189   
4   8.476733  161.047311  170.333600  ...  3.986667  0.693147  4.42709   

    piPC5.1     piPC6   piPC6.1     piPC7   piPC7.1  piPC8  pi

In [7]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#knn
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_train=a_X_train.dropna(axis=1)
    a_X_test=a_X_test.dropna(axis=1)
    
    param_knn={'n_neighbors':[1,2,3,4,5,6,7,8,9,10], 'p':[1,2,3,4,5]}
    reg_knn = GridSearchCV(KNeighborsRegressor(), param_grid=param_knn, cv=5, n_jobs=2)
    reg_knn.fit(a_X_train,y_train['Yield'])
    reg_best = reg_knn.best_estimator_
    y_pred1 = reg_best.predict(a_X_train)
    y_pred2 = reg_best.predict(a_X_test)

    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    parametors = reg_knn.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_knn.best_score_
    data_cv_score.append(cv_score)
 
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_knn.best_params_)
    print("Best Model Score:",reg_knn.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.7527900886464024
R2_test: 0.5616277396559676
----------------------
seed: 1
Best Model Parameter: {'n_neighbors': 4, 'p': 1}
Best Model Score: 0.7096420660048605
R2_test: 0.47233298889206177
----------------------
seed: 2
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.637988516687621
R2_test: 0.6745690632176091
----------------------
seed: 3
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.6169296751527987
R2_test: 0.7009645813869343
----------------------
seed: 4
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.6527959348162862
R2_test: 0.657514991704448
----------------------
seed: 5
Best Model Parameter: {'n_neighbors': 4, 'p': 1}
Best Model Score: 0.690483808813145
R2_test: 0.5341281560783524
----------------------
seed: 6
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.6727476877239644
R2_test: 0.6576

In [8]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['MSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])
        
data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd], 
                     axis=1, join='inner')
data_all.loc['mean'] = data_all.mean()
print(data_all)
data_all.to_csv('../../score/Mordred/knn_score.csv')

      r2_train   MSE_train  MAE_train   r2_test   RMSE_test   MAE_test  \
0     0.798360  148.662530   7.968085  0.561628  573.369048  18.119048   
1     0.779479  199.418883   9.686170  0.472333  511.229167  17.130952   
2     0.749167  232.045508   9.556738  0.674569  314.711640  14.095238   
3     0.714620  266.177160  10.401216  0.700965  286.998056  12.265306   
4     0.704719  254.560505  10.851064  0.657515  364.172619  12.773810   
5     0.738432  211.801862   9.962766  0.534128  566.595238  17.023810   
6     0.764787  193.033435   9.367781  0.657681  411.966958  13.455782   
7     0.740184  248.513678  10.215805  0.662274  306.151603  13.292517   
8     0.754180  216.350399   9.563830  0.565081  429.802083  14.273810   
9     0.774147  228.889362  10.940426  0.721013  196.590476  10.895238   
mean  0.751807  219.945332   9.851388  0.620719  396.158689  14.332551   

      cv_score  
0     0.752790  
1     0.709642  
2     0.637989  
3     0.616930  
4     0.652796  
5     0.6